[Here is Wikipedia's list of all HTTP response codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
+ 100's: information
+ 200's: success           
+ 300's: redirects
+ 400's + 500's: errors

Perhaps familiar:  404 <br>
Especially fun:  418


In [8]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
import matplotlib.animation as animation
from matplotlib import style
import datetime as dt
# style.use('seaborn-v0_8-dark-palette')

import requests;


In [10]:
# %matplotlib qt
# NECESSARY SO JUPYTER SHOWS LIVE PLOT!!!

In [11]:

def jsonDataRead():
    url = "http://172.28.90.134/"
    page = "get"
    param = "read-data"
    message = ""
    query = url+page+"?"+param+message
    result = requests.get(query)
    # contents = result.text
    json_contents = result.json()
    timestamp = dt.datetime.now().strftime('%H:%M:%S.%f')
    time_data = [timestamp, json_contents]
    # print(json_contents)
    return time_data


In [12]:
jsonDataRead()

['18:40:51.807486', {'time': 281600, 'button status': 1, 'thermistor': 4095}]

In [14]:
# This function is called periodically from FuncAnimation
def animate(i, xs, ys, xs1, ys1, squish_bool):
    

    # Read button status from ESP32
    timestamp, data = jsonDataRead()
    # print(buttonStatus)

    # Add read time and button status to lists
    xs.append(timestamp)
    ys.append(data["button status"])

    xs1.append(timestamp)
    ys1.append(data['thermistor'])

     
    # plt.locator_params(axis='x', nbins=6)

    if squish_bool:
        pass
    else:
        # Limit x and y lists to 20 items, optional
        xs = xs[-20:]
        ys = ys[-20:]

        xs1 = xs1[-20:]
        ys1 = ys1[-20:]

    # Draw x and y lists
    ax.clear()
    ax.plot(xs, ys)

    ax1.clear()
    ax1.plot(xs1, ys1)
    

    # xticks = ax.get_xticks()
    # ax.set_xticks(xticks[::len(xticks) // 4]) # set new tick positions
    # ax.tick_params(axis='x', rotation=30) # set tick rotation
    # ax.margins(x=0) # set tight margins

    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax1.xaxis.set_major_locator(plt.MaxNLocator(5))

    # Format plot
    plt.xticks(rotation=45, ha='right')
    plt.subplots_adjust(bottom=0.30)
    plt.title('TMP102 Temperature over Time')
    plt.ylabel('Temperature (deg C)')



In [20]:
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# style.use('seaborn-v0_8-dark-palette')



# Create figure for plotting
fig = plt.figure()
ax = fig.add_subplot(2, 1, 1)
ax1 = fig.add_subplot(2, 1, 2)
xs = []
ys = []
xs1 = []
ys1 = []


# Initialize communication with TMP102

# Set up plot to call animate() function periodically

ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys, xs1, ys1, True), interval=50)
plt.show()

/var/folders/t2/8szqbhws4zsc41hzthbc_6bw0000gn/T/ipykernel_18288/1048997901.py:22: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys, xs1, ys1, True), interval=50)


Traceback (most recent call last):
  File "/Users/allisonmarten/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/Users/allisonmarten/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/allisonmarten/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/allisonmarten/anaconda3/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/Users/allisonmarten/anaconda3/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/Users/allisonmarten/anaconda3/lib/python3.10/http/client.py", line 287, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconn